## Eval

In [92]:
import os, subprocess, re, ast, pickle

In [93]:
kitti_pointpillars_model_names = [
    "pointpillars_kitti_eloss33",
    "pointpillars_kitti_eloss66",
    "pointpillars_kitti_eloss99"
]
kitti_voxelnet_model_names = [
    "voxelnet_kitti_eloss50",
    "voxelnet_kitti_eloss100"
]

In [94]:
kitti_pointpillars_eval_results = []
for i, model_name in enumerate(kitti_pointpillars_model_names):
    model_root = f"work_dirs/{model_name}"
    config_file_name = f"{model_name}.py"
    config_file_path = os.path.join(model_root, config_file_name)
    
    if os.path.exists(os.path.join(model_root, "eval_result.pickle")):
        print(model_name)
        with open(os.path.join(model_root, "eval_result.pickle"), "rb") as f:
            eval_result = pickle.load(f)
    else:
        print(model_name)
        eval_result = []
        for epoch in range(81+5*i, 86+5*i):
            checkpoint_path = os.path.join(model_root, f"epoch_{epoch}.pth")
            command = f"tools/dist_test.sh {config_file_path} {checkpoint_path} 8 --eval mAP"
            process = subprocess.Popen(command,shell=True,stdout=subprocess.PIPE)
            out = process.stdout.read().decode("gbk")
            pattern = re.compile("{'.*:.*}")
            dict_string = pattern.search(out).group()
            eval_dict = ast.literal_eval(dict_string.replace("nan", "None"))
            eval_result.append(eval_dict)

        with open(os.path.join(model_root, "eval_result.pickle"), "wb") as f:
            pickle.dump(eval_result, f)
            
    kitti_pointpillars_eval_results.append(eval_result)

pointpillars_kitti_eloss33
pointpillars_kitti_eloss66
pointpillars_kitti_eloss99


In [80]:
kitti_voxelnet_eval_results = []
for i, model_name in enumerate(kitti_voxelnet_model_names):
    model_root = f"work_dirs/{model_name}"
    config_file_name = f"{model_name}.py"
    config_file_path = os.path.join(model_root, config_file_name)
    
    if os.path.exists(os.path.join(model_root, "eval_result.pickle")):
        print(model_name)
        with open(os.path.join(model_root, "eval_result.pickle"), "rb") as f:
            eval_result = pickle.load(f)
    else:
        print(model_name)
        with open("work_dirs/voxelnet_kitti_eloss50/20220722_105331.log.json", "r") as f:
        # with open("work_dirs/voxelnet_kitti_eloss100/20220722_114805.log.json", "r") as f:
            content = f.read()
        pattern = re.compile("{\"mode.*:.*}")
        dict_string_iter = pattern.finditer(content)

        eval_result = []
        for dict_string in dict_string_iter:
            eval_dict = ast.literal_eval(dict_string.group().replace("nan", "None"))
            if "mode" in eval_dict and eval_dict["mode"] == "val":
                eval_result.append(eval_dict)

        with open(os.path.join(model_root, "eval_result.pickle"), "wb") as f:
            pickle.dump(eval_result, f)
            
    kitti_voxelnet_eval_results.append(eval_result)

voxelnet_kitti_eloss50
voxelnet_kitti_eloss100


In [81]:
import re, ast, pandas#, seaborn
# import matplotlib.pyplot as plt

In [97]:
kitti_pointpillars_eval_dfs = list(map(lambda x : pandas.DataFrame.from_dict(x), kitti_pointpillars_eval_results))
kitti_voxelnet_eval_dfs = list(map(lambda x : pandas.DataFrame.from_dict(x), kitti_voxelnet_eval_results))

In [102]:
for eval_df in kitti_pointpillars_eval_dfs:
    print(eval_df[["KITTI/Car_3D_AP40_moderate_strict"]])

   KITTI/Car_3D_AP40_moderate_strict
0                          75.816906
1                          79.162018
2                          78.992568
3                          78.417320
4                          79.545259
   KITTI/Car_3D_AP40_moderate_strict
0                          81.698426
1                          79.071148
2                          78.031822
3                          80.003314
4                          82.613027
   KITTI/Car_3D_AP40_moderate_strict
0                          73.109124
1                          79.132848
2                          79.301024
3                          81.200176
4                          77.562604


In [87]:
for eval_df in kitti_voxelnet_eval_dfs:
    print(eval_df[["epoch", "KITTI/Car_3D_AP40_moderate_strict"]])

   epoch  KITTI/Car_3D_AP40_moderate_strict
0     36                           66.32669
1     37                           79.29933
2     38                           76.46516
3     39                           78.90461
4     40                           77.95029
5     41                           78.30299
6     42                           78.63699
7     43                           78.91905
8     44                           80.78463
9     45                           81.59947
   epoch  KITTI/Car_3D_AP40_moderate_strict
0     46                           80.27750
1     47                           82.44411
2     48                           82.33411
3     49                           82.32945
4     50                           79.90354


## Test

In [112]:
for model_name in kitti_pointpillars_model_names+kitti_voxelnet_model_names:
    model_root = f"work_dirs/{model_name}"
    config_file_name = f"{model_name}.py"
    print(model_name)
        
    config_file_path = os.path.join(model_root, config_file_name)
    
    checkpoint_path = os.path.join(model_root, f"final.pth")
    
    test_dir = f"results/kitti-car/{model_name}"
    
    # command_1 = f"mkdir -p {test_dir}"
    command_2 = f"./tools/dist_test.sh {config_file_path} {checkpoint_path} 8 --out {os.path.join(test_dir, 'results_eval.pkl')} --format-only --eval-options 'pklfile_prefix={os.path.join(test_dir, 'kitti_results')}' 'submission_prefix={os.path.join(test_dir, 'kitti_results')}'"
    
    # os.system(command_1)
    os.system(command_2)

pointpillars_kitti_eloss33


/opt/conda/lib/python3.8/site-packages/torch/distributed/launch.py:178: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torch.distributed.run.
Note that --use_env is set by default in torch.distributed.run.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
/user-data/mmdetection3d/mmdet3d/models/backbones/mink_resnet.py:9: UserWarning: Please follow `getting_started.md` to install MinkowskiEngine.`
  warnings.warn(
/user-data/mmdetection3d/mmdet3d/models/backbones/mink_resnet.py:9: UserWarning: Please fo

load checkpoint from local path: work_dirs/pointpillars_kitti_eloss33/final.pth
load checkpoint from local path: work_dirs/pointpillars_kitti_eloss33/final.pth
load checkpoint from local path: work_dirs/pointpillars_kitti_eloss33/final.pth
load checkpoint from local path: work_dirs/pointpillars_kitti_eloss33/final.pth
load checkpoint from local path: work_dirs/pointpillars_kitti_eloss33/final.pth
load checkpoint from local path: work_dirs/pointpillars_kitti_eloss33/final.pth
load checkpoint from local path: work_dirs/pointpillars_kitti_eloss33/final.pth
load checkpoint from local path: work_dirs/pointpillars_kitti_eloss33/final.pth
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 7520/7518, 379.9 task/s, elapsed: 20s, ETA:     0s
writing results to results/kitti-car/pointpillars_kitti_eloss33/results_eval.pkl

Converting prediction to KITTI format
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 7518/7518, 669.7 task/s, elapsed: 11s, ETA:     0s
Result is saved to results/kitti-car/pointpillars_kitti_eloss33/kitti_results.pk

/opt/conda/lib/python3.8/site-packages/torch/distributed/launch.py:178: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torch.distributed.run.
Note that --use_env is set by default in torch.distributed.run.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
/user-data/mmdetection3d/mmdet3d/models/backbones/mink_resnet.py:9: UserWarning: Please follow `getting_started.md` to install MinkowskiEngine.`
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/mmdet/utils/setup_env.py:48: UserWarning: Setting M

load checkpoint from local path: work_dirs/pointpillars_kitti_eloss66/final.pth
load checkpoint from local path: work_dirs/pointpillars_kitti_eloss66/final.pth
load checkpoint from local path: work_dirs/pointpillars_kitti_eloss66/final.pth
load checkpoint from local path: work_dirs/pointpillars_kitti_eloss66/final.pth
load checkpoint from local path: work_dirs/pointpillars_kitti_eloss66/final.pth
load checkpoint from local path: work_dirs/pointpillars_kitti_eloss66/final.pth
load checkpoint from local path: work_dirs/pointpillars_kitti_eloss66/final.pth
load checkpoint from local path: work_dirs/pointpillars_kitti_eloss66/final.pth
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 7520/7518, 397.3 task/s, elapsed: 19s, ETA:     0s
writing results to results/kitti-car/pointpillars_kitti_eloss66/results_eval.pkl

Converting prediction to KITTI format
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 7518/7518, 660.4 task/s, elapsed: 11s, ETA:     0s
Result is saved to results/kitti-car/pointpillars_kitti_eloss66/kitti_results.pk

...

load checkpoint from local path: work_dirs/pointpillars_kitti_eloss99/final.pth
load checkpoint from local path: work_dirs/pointpillars_kitti_eloss99/final.pth
load checkpoint from local path: work_dirs/pointpillars_kitti_eloss99/final.pth
load checkpoint from local path: work_dirs/pointpillars_kitti_eloss99/final.pth
load checkpoint from local path: work_dirs/pointpillars_kitti_eloss99/final.pth
load checkpoint from local path: work_dirs/pointpillars_kitti_eloss99/final.pth
load checkpoint from local path: work_dirs/pointpillars_kitti_eloss99/final.pth
load checkpoint from local path: work_dirs/pointpillars_kitti_eloss99/final.pth
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 7520/7518, 389.3 task/s, elapsed: 19s, ETA:     0s
writing results to results/kitti-car/pointpillars_kitti_eloss99/results_eval.pkl

Converting prediction to KITTI format
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 7518/7518, 637.9 task/s, elapsed: 12s, ETA:     0s
Result is saved to results/kitti-car/pointpillars_kitti_eloss99/kitti_results.pk

/opt/conda/lib/python3.8/site-packages/torch/distributed/launch.py:178: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torch.distributed.run.
Note that --use_env is set by default in torch.distributed.run.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
/user-data/mmdetection3d/mmdet3d/models/backbones/mink_resnet.py:9: UserWarning: Please follow `getting_started.md` to install MinkowskiEngine.`
  warnings.warn(
/user-data/mmdetection3d/mmdet3d/models/backbones/mink_resnet.py:9: UserWarning: Please fo

load checkpoint from local path: work_dirs/voxelnet_kitti_eloss50/final.pth
load checkpoint from local path: work_dirs/voxelnet_kitti_eloss50/final.pth
load checkpoint from local path: work_dirs/voxelnet_kitti_eloss50/final.pth
load checkpoint from local path: work_dirs/voxelnet_kitti_eloss50/final.pth
load checkpoint from local path: work_dirs/voxelnet_kitti_eloss50/final.pth
load checkpoint from local path: work_dirs/voxelnet_kitti_eloss50/final.pth
load checkpoint from local path: work_dirs/voxelnet_kitti_eloss50/final.pth
load checkpoint from local path: work_dirs/voxelnet_kitti_eloss50/final.pth
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 7520/7518, 219.5 task/s, elapsed: 34s, ETA:     0s
writing results to results/kitti-car/voxelnet_kitti_eloss50/results_eval.pkl

Converting prediction to KITTI format
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 7518/7518, 660.9 task/s, elapsed: 11s, ETA:     0s
Result is saved to results/kitti-car/voxelnet_kitti_eloss50/kitti_results.pkl.
voxelnet_kitti_eloss100


/opt/conda/lib/python3.8/site-packages/torch/distributed/launch.py:178: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torch.distributed.run.
Note that --use_env is set by default in torch.distributed.run.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
/user-data/mmdetection3d/mmdet3d/models/backbones/mink_resnet.py:9: UserWarning: Please follow `getting_started.md` to install MinkowskiEngine.`
  warnings.warn(
/user-data/mmdetection3d/mmdet3d/models/backbones/mink_resnet.py:9: UserWarning: Please fo

load checkpoint from local path: work_dirs/voxelnet_kitti_eloss100/final.pth
load checkpoint from local path: work_dirs/voxelnet_kitti_eloss100/final.pth
load checkpoint from local path: work_dirs/voxelnet_kitti_eloss100/final.pth
load checkpoint from local path: work_dirs/voxelnet_kitti_eloss100/final.pth
load checkpoint from local path: work_dirs/voxelnet_kitti_eloss100/final.pth
load checkpoint from local path: work_dirs/voxelnet_kitti_eloss100/final.pth
load checkpoint from local path: work_dirs/voxelnet_kitti_eloss100/final.pth
load checkpoint from local path: work_dirs/voxelnet_kitti_eloss100/final.pth
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 7520/7518, 229.2 task/s, elapsed: 33s, ETA:     0s
writing results to results/kitti-car/voxelnet_kitti_eloss100/results_eval.pkl

Converting prediction to KITTI format
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 7518/7518, 689.9 task/s, elapsed: 11s, ETA:     0s
Result is saved to results/kitti-car/voxelnet_kitti_eloss100/kitti_results.pkl.


In [120]:
model_name = "pointpillars_kitti_eloss33"
model_root = f"work_dirs/{model_name}"
config_file_name = f"{model_name}.py"
config_file_path = os.path.join(model_root, config_file_name)
checkpoint_path = "checkpoints/pointpillars_kitti/hv_pointpillars_secfpn_6x8_160e_kitti-3d-car_20220331_134606-d42d15ed.pth"
test_dir = f"results/kitti-car/pointpillars_kitti_eloss0"

# command_1 = f"mkdir -p {test_dir}"
command_2 = f"./tools/dist_test.sh {config_file_path} {checkpoint_path} 8 --out {os.path.join(test_dir, 'results_eval.pkl')} --format-only --eval-options 'pklfile_prefix={os.path.join(test_dir, 'kitti_results')}' 'submission_prefix={os.path.join(test_dir, 'kitti_results')}'"

# os.system(command_1)
os.system(command_2)

/opt/conda/lib/python3.8/site-packages/torch/distributed/launch.py:178: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torch.distributed.run.
Note that --use_env is set by default in torch.distributed.run.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
/user-data/mmdetection3d/mmdet3d/models/backbones/mink_resnet.py:9: UserWarning: Please follow `getting_started.md` to install MinkowskiEngine.`
  warnings.warn(
/user-data/mmdetection3d/mmdet3d/models/backbones/mink_resnet.py:9: UserWarning: Please fo

load checkpoint from local path: checkpoints/pointpillars_kitti/hv_pointpillars_secfpn_6x8_160e_kitti-3d-car_20220331_134606-d42d15ed.pth
load checkpoint from local path: checkpoints/pointpillars_kitti/hv_pointpillars_secfpn_6x8_160e_kitti-3d-car_20220331_134606-d42d15ed.pth
load checkpoint from local path: checkpoints/pointpillars_kitti/hv_pointpillars_secfpn_6x8_160e_kitti-3d-car_20220331_134606-d42d15ed.pth
load checkpoint from local path: checkpoints/pointpillars_kitti/hv_pointpillars_secfpn_6x8_160e_kitti-3d-car_20220331_134606-d42d15ed.pth
load checkpoint from local path: checkpoints/pointpillars_kitti/hv_pointpillars_secfpn_6x8_160e_kitti-3d-car_20220331_134606-d42d15ed.pth
load checkpoint from local path: checkpoints/pointpillars_kitti/hv_pointpillars_secfpn_6x8_160e_kitti-3d-car_20220331_134606-d42d15ed.pth
load checkpoint from local path: checkpoints/pointpillars_kitti/hv_pointpillars_secfpn_6x8_160e_kitti-3d-car_20220331_134606-d42d15ed.pth
load checkpoint from local path: c

0

In [121]:
model_name = "voxelnet_kitti_eloss50"
model_root = f"work_dirs/{model_name}"
config_file_name = f"{model_name}.py"
config_file_path = os.path.join(model_root, config_file_name)
checkpoint_path = "checkpoints/second_kitti/hv_second_secfpn_6x8_80e_kitti-3d-car_20200620_230238-393f000c.pth"
test_dir = f"results/kitti-car/voxelnet_kitti_eloss0"

# command_1 = f"mkdir -p {test_dir}"
command_2 = f"./tools/dist_test.sh {config_file_path} {checkpoint_path} 8 --out {os.path.join(test_dir, 'results_eval.pkl')} --format-only --eval-options 'pklfile_prefix={os.path.join(test_dir, 'kitti_results')}' 'submission_prefix={os.path.join(test_dir, 'kitti_results')}'"

# os.system(command_1)
os.system(command_2)

/opt/conda/lib/python3.8/site-packages/torch/distributed/launch.py:178: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torch.distributed.run.
Note that --use_env is set by default in torch.distributed.run.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
/user-data/mmdetection3d/mmdet3d/models/backbones/mink_resnet.py:9: UserWarning: Please follow `getting_started.md` to install MinkowskiEngine.`
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/mmdet/utils/setup_env.py:48: UserWarning: Setting M

load checkpoint from local path: checkpoints/second_kitti/hv_second_secfpn_6x8_80e_kitti-3d-car_20200620_230238-393f000c.pth
load checkpoint from local path: checkpoints/second_kitti/hv_second_secfpn_6x8_80e_kitti-3d-car_20200620_230238-393f000c.pth
load checkpoint from local path: checkpoints/second_kitti/hv_second_secfpn_6x8_80e_kitti-3d-car_20200620_230238-393f000c.pth
load checkpoint from local path: checkpoints/second_kitti/hv_second_secfpn_6x8_80e_kitti-3d-car_20200620_230238-393f000c.pth
load checkpoint from local path: checkpoints/second_kitti/hv_second_secfpn_6x8_80e_kitti-3d-car_20200620_230238-393f000c.pth
load checkpoint from local path: checkpoints/second_kitti/hv_second_secfpn_6x8_80e_kitti-3d-car_20200620_230238-393f000c.pth
load checkpoint from local path: checkpoints/second_kitti/hv_second_secfpn_6x8_80e_kitti-3d-car_20200620_230238-393f000c.pth
load checkpoint from local path: checkpoints/second_kitti/hv_second_secfpn_6x8_80e_kitti-3d-car_20200620_230238-393f000c.pth


0